In [1]:
!pip install 'biopandas'
!pip install scipy
!pip install scikit-learn
!pip install biopython

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip


In [2]:
import argparse
import collections
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
import subprocess
import time
from biopandas.pdb import PandasPdb
from Bio.PDB import PDBParser
from Bio.PDB.SASA import ShrakeRupley
from sklearn.neighbors import KDTree
from collections import defaultdict
import csv
import pandas as pd

In [3]:
data_path = "/home/rnomura/research/analysis/analysis/pdb_files/"
itp_path = "/home/rnomura/research/analysis/analysis/itp_files/"
process_data_path = "/home/rnomura/research/analysis/analysis/pdb_new_files/"
research_path = "/home/rnomura/research/analysis/analysis/analysis_program/csv/"
pic_path = "/home/rnomura/research/analysis/analysis/analysis_program/pic_stock/"

- itpファイルを読み取る。
- 原子の箇所をitpの種類で置き換える(biopandas使う。)R, S, Tで、最初にSもTもなかったらR、SならS、TならT
- そのR, T, Sをradii_dictで辞書配列でbiopythonのSASAに渡す。
- SASAのlevel=Rで露出表面積を計算する。bio.pdb.sasa.Shrakerupley(proe_radiusも粗視化水分子の半径で)
- これは絶対SASAらしいので、それを踏まえてmolpatchを書き換える。

itpファイルをと読み取り、原子の箇所をR:RR, S:SS, T:TTで置き換える!\
biopandas(https://github.com/BioPandas/biopandas/blob/main/biopandas/pdb/pandas_pdb.py)

In [4]:
def itp_to_pdb(keyword):
    with open(itp_path + keyword + '.itp','r') as f:
      memo_lines = []
      flag = 0
      for line in f:    
        if '[ atoms ]' in line:
          flag = 1
          continue
        elif '[ position_restraints ]' in line:
          flag = 0
          break
        if flag:
          memo_lines.append(line.rstrip('\n'))
      #memo_linesの余計なものを除去
      memo_beads_type = []
      memo_charge = []
      for line in memo_lines[:-1]:
        if line[4:5] == 'S':
          memo_beads_type.append('SS')
          memo_charge.append(float(line[-4:-1]))
        elif line[4:5] == 'T':
          memo_beads_type.append('TT')
          memo_charge.append(float(line[-4:-1]))
        elif line == '':
          pass
        else:
          memo_beads_type.append('RR')
          memo_charge.append(float(line[-4:-1]))
    return pd.Series(memo_beads_type).values, memo_charge

In [5]:
atom_list, charge_list = itp_to_pdb('minus1')#minus1.itp

In [6]:
def make_new_pdb(keyword):
    dic_r = {'RR':2.35,'SS':2.05,'TT':1.7}
    path = data_path + keyword + '.pdb'
    pdb_data_s = PandasPdb().read_pdb(path)
    pdb_data_s.df['ATOM']['atom_name'] = atom_list
    pdb_data_s.to_pdb(path=process_data_path + keyword + '.pdb')

In [7]:
make_new_pdb('minus1_cg')#cgモデルの方はminus1_cg.pdb

R, T, Sをradii_dictで辞書配列でbiopythonのSASAに渡す。\
sasa(https://biopython.org/docs/dev/api/Bio.PDB.SASA.html)\
読み込むdataはprocess_data_pathにあるファイル\
SASAのlevel=Rで露出表面積を計算する。bio.pdb.sasa.Shrakerupley(proe_radiusも粗視化水分子の半径で)

In [20]:
p = PDBParser(QUIET=1)
struct = p.get_structure('minus1_cg', process_data_path + 'minus1_cg.pdb')
sr = ShrakeRupley(probe_radius=1.4, radii_dict={"RR":2.35, "SS":2.05, "TT":1.7})
sr.compute(struct, level="R")
#sr.compute(struct, level="S")
#struct.sasa

In [21]:
#for chain in struct[0]:
    #for residue in chain:
        #pass
        #print(residue.sasa)

In [22]:
#dssp_arr = [residue.sasa for chain in struct[0] for residue in chain]
#len(dssp_arr)

In [23]:
def three_to_one(residue_name):
    amino_acids = {
        "ALA": "A", "CYS": "C", "ASP": "D", "GLU": "E",
        "PHE": "F", "GLY": "G", "HIS": "H", "ILE": "I",
        "LYS": "K", "LEU": "L", "MET": "M", "ASN": "N",
        "PRO": "P", "GLN": "Q", "ARG": "R", "SER": "S",
        "THR": "T", "VAL": "V", "TRP": "W", "TYR": "Y"
    }
    # 3文字コードを1文字コードに変換
    return amino_acids.get(residue_name.upper(), "X")
    
residue_arr = [three_to_one(residue.get_resname()) for chain in struct[0] for residue in chain]
#residue_arr

これは絶対SASAらしいので、それを踏まえてmolpatchを書き換える。

In [24]:
from Bio.PDB import PDBParser, MMCIFParser, Selection
from Bio.PDB.ResidueDepth import get_surface
from Bio.PDB.DSSP import dssp_dict_from_pdb_file
from Bio.PDB.DSSP import DSSP
from scipy.spatial import KDTree
from Bio.SeqUtils import seq1
from pprint import pprint
import networkx as nx
import numpy as np
import pandas as pd
import os

#add
def three_to_one(residue_name):
    amino_acids = {
        "ALA": "A", "CYS": "C", "ASP": "D", "GLU": "E",
        "PHE": "F", "GLY": "G", "HIS": "H", "ILE": "I",
        "LYS": "K", "LEU": "L", "MET": "M", "ASN": "N",
        "PRO": "P", "GLN": "Q", "ARG": "R", "SER": "S",
        "THR": "T", "VAL": "V", "TRP": "W", "TYR": "Y"
    }
    # 3文字コードを1文字コードに変換
    return amino_acids.get(residue_name.upper(), "X")

class ResiduePatch():
    """
    Parse pisite files to dict

    Attributes
    ----------
    ids : list
        residue ids
    file: str
        pdb file
    dssp: dict
        dssp dict
    dssp_dict_keys: dict
        dssp dict keys
    """

    def __init__(self, ids, dssp):
        self.ids = ids
        #print(ids)
        self.dssp = dssp
        #print(dssp_dict_keys)
        #self.dssp_dict_keys = dssp_dict_keys

    def get_ids(self):
        return self.ids

    def size(self):
        #absolute_asa = {"G":104,"M":224,"A":129,"P":159,"V":174,"F":240,"L":201,"I":197,"W":285}
        #return sum(self.dssp[i[-2:]][2] for i in self.ids)#edited
        #print(('minus1', 0, 'A', (' ', 46, ' '))[-1][1])#idから上手く
        return sum(self.dssp[i[-1][1] - 1] for i in self.ids)
    def residues(self):
        #return [self.dssp[i[-2:]][0] for i in self.ids]
        residue_list = [three_to_one(residue.get_resname()) for chain in struct[0] for residue in chain]
        return [residue_list[i[-1][1] - 1] for i in self.ids]#ここで上手くアミノ酸配列を与えないといけない、ここを直接与えるかorどこからか引っ張ってくる>
    def patch_length(self):
        return len(self.ids)

    def residue_on_surface(self):#unused
        """
        Get all residues accessible from the surface

        Return
        ------
        list
            residue ids
        """
        residue_list = []
        for x in self.dssp_dict_keys:
            if x[1][0] != ' ' or  x[1][2] != ' ':
                continue
            if self.dssp[x][2] > 0:
                residue_list.append(x)
        #print(residue_list)
        return residue_list
        
class ProteinPatch():

    def __init__(self, id, file, residues_in_patch, r=1.25, msms='msms -density 1.5'):
        if file.endswith('.pdb'):
            parser = PDBParser(QUIET=1)    
        elif file.endswith('.cif'):
            parser = MMCIFParser(QUIET=1)
        structure = parser.get_structure(id, file)
        self.model = structure[0]
        self.r = r
        if not msms.startswith('msms'):
            msms = 'msms ' + msms
        self.msms = msms
        self.residues_in_patch = residues_in_patch
        #self.dssp_dict = DSSP(self.model, file)
        #p = PDBParser(QUIET=1)
        #struct = p.get_structure('minus1_cg', process_data_path + 'minus1_cg.pdb')
        sr = ShrakeRupley(probe_radius=2.35, radii_dict={"RR":2.35, "SS":2.05, "TT":1.7})
        sr.compute(structure, level="R")
        self.dssp_arr = [residue.sasa for chain in struct[0] for residue in chain]
        #self.dssp_dict_keys = self.dssp_dict.keys()
        #print(self.dssp_dict[self.dssp_dict_keys[2]][3])#dict[key][3]
        #self.dssp_dict, self.dssp_dict_keys =  dssp_dict_from_pdb_file(file)
        self.G = self.dot_cloud_graph()
        self.G = self.patch_network(self.G)
        self.patches = self.create_patches()

    def dot_cloud_graph(self):
        """
        Create a dotted cloud of the proteins surface and label the dots
        hydrophobic or hydrophilic
        ...

        Return
        ------
        networkx.Graph
            Graph with nodes labeled as hydrophobic or not
        """
        surface_points = get_surface(self.model, MSMS=self.msms)
        residue_list = [r for r in Selection.unfold_entities(self.model, "R") if seq1(r.get_resname()) != 'X']
        center_vectices = [self._sidechain_center(r.get_atoms()) for r in residue_list]

        T = KDTree(center_vectices)
		
        closest_residues = T.query(surface_points, k=1)[1]
        #print(closet_residues)
        G = nx.Graph()
        for node, coordinates in enumerate(surface_points):
            G.add_node(node)
            G.nodes[node]['selected'] = 0
            closest_residue = residue_list[closest_residues[node]]
            if seq1(closest_residue.get_resname()) in self.residues_in_patch:
                G.nodes[node]['selected'] = 1
            G.nodes[node]['surface_vector_pos'] = coordinates
            G.nodes[node]['closest_residue_id'] = closest_residue.get_full_id()
            G.nodes[node]['closest_residue_aa'] = seq1(closest_residue.get_resname())
        return G

    def patch_network(self, G):
        """
        Create a edges between hydrophobic nodes if they are within r
        hydrophobic or hydrophilic
        ...

        Attributes
        ----------
        G : networkx.Graph
            Graph with nodes labeled as hydrophobic or not

        Return
        ------
        networkx.Graph
            Graph with nodes labeled as hydrophobic and edges between nodes within range r
        """
        node_list = [i for i in G.nodes if G.nodes[i]['selected']]
        x = [G.nodes[i]['surface_vector_pos'] for i in G.nodes if G.nodes[i]['selected']]
        T = KDTree(x)
        pairs = T.query_pairs(self.r)
        G.add_edges_from([(node_list[x[0]],node_list[x[1]]) for x in pairs])
        return G

    def largest_patch(self):
        """
        get largest patch

        Return
        ------
        networkx.Graph
            Graph of the largest patch
        """
        largest_patch = max(self.patches, key=(lambda x: x.size()))
        
        return largest_patch

    def create_patches(self):
        """
        Get the components of the graph

        Return
        ------
        list
            list of Graph components where an item is a patch
        """
        patched_G = self.patch_network(self.G)
        components = [patched_G.subgraph(c) for c in nx.connected_components(patched_G)] #nx.connected_component_subgraphs(patched_G)
        patch_dict = []
        for component in components:
            if len(component.nodes) <= 1:
                continue
            residue_ids_in_patch = list(set([component.nodes[i]['closest_residue_id'] for i in component.nodes]))
            patch_dict.append(ResiduePatch(residue_ids_in_patch, self.dssp_arr))
        #print([i.size() for i in patch_dict])
        return sorted(patch_dict, key=(lambda x: x.size()), reverse=True)

    def _sidechain_center(self, atoms):
        vectors = [atom.get_vector().get_array() for atom in atoms]
        center = np.array(vectors).mean(axis=0)
        return center
def calculate_patches(infile, residues, plot, output_dir):
    """ Calculate patches on a protein surface """

    filename = os.path.basename(infile)
    outfile = filename.split('.')[0] + '.csv'
    pdb_id = filename.split('.')[0].split('_')[0]

    proteinPatches = ProteinPatch(pdb_id,infile,residues)
    patches = proteinPatches.patches
    #print([i.get_ids() for i in patches])
    result_dict = {'patch_rank':[], 'protein_id':[], 'residue_id':[], 'chain':[], 'residue_type':[], 'patch_size':[]}
    for i,patch in enumerate(patches):
        residues_in_patch = patch.get_ids()
        residues = patch.residues()
        
        for j, residue_in_patch in enumerate(residues_in_patch):
            result_dict['residue_id'].append(residue_in_patch[-2:][1][1])
            result_dict['chain'].append(residue_in_patch[-2:][0])
            result_dict['patch_size'].append(patch.size())
            result_dict['patch_rank'].append(i)
            result_dict['residue_type'].append(residues[j])
            result_dict['protein_id'].append(pdb_id)
            

    pd.DataFrame(result_dict).to_csv(os.path.join(output_dir, "patches_" + outfile), index=False)

    if plot:
        outfig = infile.split('.')[0] + '.png'
        proteinPatches.plot_largest_patches(outfig)

In [25]:
file = process_data_path + 'minus1_cg.pdb'
output_dir = '../output/'
residues = ["G","M","A","P","V","F","L","I","W"]
calculate_patches(infile=file, residues=residues, plot=False, output_dir=output_dir)

/home/rnomura/.local/lib/python3.10/site-packages/Bio/PDB/ResidueDepth.py:490: BiopythonWarning: TT:ALA not in radii library.
  warnings.warn(f"{at_name}:{resname} not in radii library.", BiopythonWarning)
/home/rnomura/.local/lib/python3.10/site-packages/Bio/PDB/ResidueDepth.py:490: BiopythonWarning: TT:PHE not in radii library.
  warnings.warn(f"{at_name}:{resname} not in radii library.", BiopythonWarning)
/home/rnomura/.local/lib/python3.10/site-packages/Bio/PDB/ResidueDepth.py:490: BiopythonWarning: TT:CYS not in radii library.
  warnings.warn(f"{at_name}:{resname} not in radii library.", BiopythonWarning)
/home/rnomura/.local/lib/python3.10/site-packages/Bio/PDB/ResidueDepth.py:490: BiopythonWarning: TT:TYR not in radii library.
  warnings.warn(f"{at_name}:{resname} not in radii library.", BiopythonWarning)
/home/rnomura/.local/lib/python3.10/site-packages/Bio/PDB/ResidueDepth.py:490: BiopythonWarning: TT:SER not in radii library.
  warnings.warn(f"{at_name}:{resname} not in radii